In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # What to do with the URL.  Here, we tell it to download all the code and save
    # it to the mainpage.html file
    def parse(self, response):
        with open('mainpage.html', 'wb') as f:
            f.write(response.body)


# Instantiate our crawler.
process = CrawlerProcess()

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()

2018-08-24 10:51:00 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-08-24 10:51:00 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.0, w3lib 1.19.0, Twisted 17.5.0, Python 3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2o  27 Mar 2018), cryptography 2.2.2, Platform Darwin-16.7.0-x86_64-i386-64bit
2018-08-24 10:51:00 [scrapy.crawler] INFO: Overridden settings: {}
2018-08-24 10:51:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-08-24 10:51:00 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermid

In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://lang-8.com/2581/journals',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        links = response.xpath('//a[contains(@href, "image")]')
        links.extract()
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('section[@class="vertical-spaced journals_flex floated_on_right"]'):
            
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'name': article.xpath('a/text()').extract(),
                'text': article.xpath('span[@class="journal body"]/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'esl4.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')


Success!


In [ ]:
import pandas as pd

In [ ]:
pd.read_json('esl.json')

# Checking the output

Did it work? We turned off logging in the example above, so the best way to check is to look at our output JSON file: `firstpage.json`.

**Note**: if you run the code more than once in a row, Scrapy will try to append the new scraped information to the end of the old JSON file. This will cause the JSON file to bug out because it will have an "end of file" notation in the middle of the file, so make sure to delete your JSON file before you re-run this or you'll get a "trailing file" error.

We can take a look at the raw json file (which is pretty human-readable) in your favorite text editor, or load it into Python as a data frame:

In [ ]:
import pandas as pd

firstpage = pd.read_json('firstpage.json', orient='records')
print(firstpage.shape)
print(firstpage.head())

Success!  Ten rows (representing ten entries) each with date, name, tags, and text.  


## Recursion

Now that we have a scraper that can pull the information we want off of a page and store it in a file, we want to run that scraper over _all_ the pages of the website. We do this using recursion – the Scrapy spider will run over a page, gather information, and then detect a link to the next page and _call itself_ on the new page.  Then the new instance of the spider will do the same thing. This will repeat until it arrives at a page with no link to a next page, or when it reaches a stopping condition we define.

Since _Everyday Sexism_ has a paginated structure (every page links to the page before it), we can use recursion to get all the entries on the site. If you wanted to scrape a website with a non-linear structure, you'd still use the recursive setup, but with different restrictions on what links the scraper should follow. Scrapy is efficient and will only scrape each page once, even if it travels to them multiple times.

You may notice we've added some additional arguments to the ´CrawlerProcess()´ command- these have to do with scraping etiquette and will be introduced in the next assignment.

Again, you'll want to restart the kernel.

In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESL"
    
    # URL(s) to start with.
    start_urls = [
        'http://lang-8.com/2581/journals',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        # Get the URL of the previous page.
        next_page = response.xpath('//div[@class="nav-previous"]/a/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        # Recursively call the spider to run on the next page, if it exists.
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)

# Tell the script how to run the crawler by passing in settings.
# The new settings have to do with scraping etiquette.          
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'data.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ESLTestScrape',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

Success!


In [ ]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf=pd.read_json('data.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())